In [47]:
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm
import scipy.ndimage
import cv2
import os
from matplotlib import pyplot as plt
import sys

# Get the home directory
home_directory = os.path.expanduser('~')

%env SITK_SHOW_COMMAND '/home/andres/Downloads/Slicer-5.4.0-linux-amd64/Slicer'
# Append the parent directory to the Python path
#sys.path.append(os.path.join(home_directory, 'Ashish/UCAN', 'UCAN-PET-CT-image-data-handling-pipeline'))
sys.path.append(os.path.join(home_directory, '/home/andres/VSCode/', 'UCAN-PET-CT-image-data-handling-pipeline'))

from Utils import utils

def display_full(x):
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 20000,
                           "display.max_colwidth", None,
                           ):
        print(x)

env: SITK_SHOW_COMMAND='/home/andres/Downloads/Slicer-5.4.0-linux-amd64/Slicer'


In [3]:
def get_proj_after_mask(img):
    
    '''

    Function to get 3D masks for CT scans to segment out the exact tissue type.

    '''

    pix_array=sitk.GetArrayFromImage(img)
    max_i, min_i=float(pix_array.max()),float(pix_array.min())

    #multiply= sitk.MultiplyImageFilter()
    #if hu_type == 'Bone' or hu_type == 'bone' or hu_type == 'B':
    bone_mask = sitk.BinaryThreshold(
    img, lowerThreshold=200, upperThreshold=max_i,insideValue=1, outsideValue=0
    )
    
    tracer_contrast_mask = sitk.BinaryThreshold(
    img, lowerThreshold=151, upperThreshold=199,insideValue=1, outsideValue=0
    )

    #bone_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
        # path= img_n + r'_{0}_image.nii'.format(modality + '_' + type)
        # save_as_gz(op_img,path)
    #elif hu_type == 'Lean Tissue' or hu_type == 'lean' or hu_type == 'LT':
    lean_mask = sitk.BinaryThreshold(
    img, lowerThreshold=-29, upperThreshold=150, insideValue=1, outsideValue=0
    )
    #lean_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))

    #elif hu_type == 'Adipose' or hu_type == 'adipose' or hu_type == 'AT':
    adipose_mask = sitk.BinaryThreshold(
    img, lowerThreshold=-199, upperThreshold=-30, insideValue=1, outsideValue=0
    )
    #adipose_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
        
    #elif hu_type == 'Air' or hu_type == 'A':
    air_mask = sitk.BinaryThreshold(
    img, lowerThreshold=min_i, upperThreshold=-191, insideValue=1, outsideValue=0
    )
    #air_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
    
    return bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask

In [4]:
#CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/npr100169878450_SUV_CT/20130412/CT.nii.gz"
CT_path = "/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/npr100169878450_SUV_CT/20130412/CT.nii.gz"

In [14]:
CT_img = sitk.ReadImage(CT_path)

In [15]:
bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img)

In [16]:
lean_mask_arr = sitk.GetArrayFromImage(lean_mask)
np.unique(lean_mask_arr)

array([0, 1], dtype=uint8)

In [17]:
lean_mask_arr.sum()

2476456

In [18]:
np.unique(lean_mask_arr, return_counts=True)

(array([0, 1], dtype=uint8), array([16332376,  2476456]))

In [19]:
voxel_spacing = [2.0364201068878174, 2.0364201068878174, 3.0]

In [20]:
# calculate lean density of lean muscle # multiply with 1e-3 to covert mm3 to cm3 (ml)  # ml * 1e-3 litres
density = lean_mask_arr.sum() * voxel_spacing[0] * voxel_spacing[1] * voxel_spacing[2] * 1e-6
density

30.809640000075532

In [21]:
# calculate bone skeleton density
bone_mask_arr = sitk.GetArrayFromImage(bone_mask)
print(np.unique(bone_mask_arr))
print(bone_mask_arr.sum())
print(np.unique(bone_mask_arr, return_counts=True))
bone_mask_arr.sum() * voxel_spacing[0] * voxel_spacing[1] * voxel_spacing[2] * 1e-6

[0 1]
262629
(array([0, 1], dtype=uint8), array([18546203,   262629]))


3.2673727873945015

In [36]:
# remove noise from lean mask
lean_mask_arr = sitk.GetArrayFromImage(lean_mask)
print(np.unique(lean_mask_arr))
print(lean_mask_arr.sum())
print(lean_mask_arr.shape)
print(np.unique(lean_mask_arr, return_counts=True))

[0 1]
2476456
(287, 256, 256)
(array([0, 1], dtype=uint8), array([16332376,  2476456]))


In [54]:
import skimage
from  scipy import ndimage

In [109]:
temp = 1 - lean_mask_arr
#temp = lean_mask_arr.copy()
#lean_mask_arr1 = skimage.morphology.remove_small_objects(temp, min_size=1500, connectivity=124, out=temp)
#footprint = skimage.morphology.octahedron(1)
#footprint = skimage.morphology.cube(5)
#lean_mask_arr1 = skimage.morphology.white_tophat(temp, footprint)
lean_mask_arr1 = skimage.morphology.binary_erosion(temp).astype(int)
lean_mask_arr1 = skimage.morphology.binary_erosion(lean_mask_arr1).astype(int)
#lean_mask_arr1 = skimage.morphology.binary_erosion(lean_mask_arr1, footprint=[(np.ones((3, 3, 3)), 2)]).astype(int)
print(np.unique(lean_mask_arr1))
print(lean_mask_arr1.sum())
print(lean_mask_arr1.shape)
print(np.unique(lean_mask_arr1, return_counts=True))
lean_mask_img1 = sitk.GetImageFromArray(lean_mask_arr1.astype(int))
print(lean_mask_img1.GetSize())
#multiply= sitk.MultiplyImageFilter()
#CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(lean_mask_img1)#, CT_img.GetPixelID())

[0 1]
1072746
(287, 256, 256)
(array([0, 1]), array([17736086,  1072746]))
(256, 256, 287)


In [105]:
CT_img.GetSize()

(256, 256, 287)

In [108]:
temp = lean_mask_arr.copy()
lean_mask_arr1 = ndimage.binary_fill_holes(temp, structure=np.ones((3,3,3)) ).astype(int)
print(np.unique(lean_mask_arr1))
print(lean_mask_arr1.sum())
print(lean_mask_arr1.shape)
print(np.unique(lean_mask_arr1, return_counts=True))
lean_mask_img1 = sitk.GetImageFromArray(lean_mask_arr1)
#multiply= sitk.MultiplyImageFilter()
#CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(lean_mask_img1)#, CT_img.GetPixelID())

[0 1]
16348356
(287, 256, 256)
(array([0, 1]), array([ 2460476, 16348356]))


NameError: name 'uint8' is not defined

In [110]:
raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_lean_projection_after_removing_noise/"

multiply= sitk.MultiplyImageFilter()
CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
utils.get_2D_projections(CT_lean, 'CT', "max", angle=90, invert_intensity= True, img_n=raw_projections_path + 'npr100169878450_SUV_CT_20130412_CT_lean_MaxIP_')
utils.get_2D_projections(CT_lean, 'CT', "min", angle=90, invert_intensity= True, img_n=raw_projections_path + 'npr100169878450_SUV_CT_20130412_CT_lean_MinIP_')


RuntimeError: Exception thrown in SimpleITK MultiplyImageFilter_Execute: /tmp/SimpleITK-build/ITK-prefix/include/ITK-5.3/itkImageToImageFilter.hxx:218:
ITK ERROR: MultiplyImageFilter(0x3e114c0): Inputs do not occupy the same physical space! 
InputImage Origin: [-2.4804688e+02, -2.4804688e+02, -7.3870001e+02], InputImage_1 Origin: [0.0000000e+00, 0.0000000e+00, 0.0000000e+00]
	Tolerance: 2.0364201e-06
InputImage Spacing: [2.0364201e+00, 2.0364201e+00, 3.0000000e+00], InputImage_1 Spacing: [1.0000000e+00, 1.0000000e+00, 1.0000000e+00]
	Tolerance: 2.0364201e-06


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-3168-36.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-3168-36.mha" "[0.28s]"
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""


In [13]:
multiply= sitk.MultiplyImageFilter()
CT_tracer_contrast = multiply.Execute(CT_img,sitk.Cast(tracer_contrast_mask,CT_img.GetPixelID()))

CT_ptype = 'mean'
SUV_ptype = 'max'
angle = 90

#raw_projections_path = "/home/ashish/Ashish/UCAN/Testing/test_new_projections/"
raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_new_projections/"

utils.get_2D_projections(CT_tracer_contrast, 'CT', CT_ptype, angle, invert_intensity= True, img_n=raw_projections_path)

Finished generating 3 - mean intensity 2D projections from the CT volume image! 


In [27]:
resampled_directory_list = []
#resampled_SUV_CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/"
resampled_SUV_CT_path = "/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/"

for dirs, subdirs, files in os.walk(resampled_SUV_CT_path):
    for file in files:
        file_path = str(os.path.join(dirs, file))
        file_path = file_path.replace('\\','/')
        resampled_directory_list.append(file_path)

resampled_directory_df = pd.DataFrame(resampled_directory_list, columns=['directory'])
resampled_directory_df[['source_directory', 'patient_directory', 'scan_date', 'SUV_CT']] = resampled_directory_df['directory'].str.rsplit(pat='/', n=3, expand=True)
resampled_directory_df[['npr', 'extra']] = resampled_directory_df['patient_directory'].str.split(pat='_', n=1, expand=True)
resampled_directory_df.drop(columns=['directory','extra', 'SUV_CT'], inplace=True)
resampled_directory_df.drop_duplicates(inplace=True)

utils.display_full(resampled_directory_df.head(2))

count = 0
# Generate the raw 2D projections
for index, row in resampled_directory_df.iterrows():
    #raw_projections_path = "/home/ashish/Ashish/UCAN/Results/test_new_proj/"
    raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_new_projections/"
    angle = 90

    CTnii_path = resampled_SUV_CT_path + str(row["patient_directory"]) + '/' + str(row['scan_date']) + '/' + 'CT.nii.gz'
    #print("CTnii_path", CTnii_path)
    
    CT_img =sitk.ReadImage(CTnii_path)
    #print("info:", CT_img.GetSize())
    #utils.save_as_gz(CT_img, raw_projections_path + 'test_CT.nii.gz')
    #sitk.WriteImage(CT_img, raw_projections_path + 'test_CT.nii.gz')

    SUVnii_path = resampled_SUV_CT_path + str(row["patient_directory"]) + '/' + str(row['scan_date']) + '/' + 'SUV.nii.gz'
    SUV_img =sitk.ReadImage(SUVnii_path)
    #print("SUVnii_path", SUVnii_path)
    #utils.save_as_gz(SUV_img, raw_projections_path + 'test_SUV.nii.gz')
    
    bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img)

    multiply= sitk.MultiplyImageFilter()
    #try:

    CT_tracer_contrast = multiply.Execute(CT_img,sitk.Cast(tracer_contrast_mask,CT_img.GetPixelID()))
 
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MaxIP_')
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "mean", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MeanIP_')
    #utils.get_2D_projections(CT_tracer_contrast, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MinIP_')

    SUV_tracer_contrast = multiply.Execute(SUV_img,sitk.Cast(tracer_contrast_mask,SUV_img.GetPixelID()))
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MaxIP_')
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "mean", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MeanIP_')
    #utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MinIP_')


    CT_bone = multiply.Execute(CT_img,sitk.Cast(bone_mask,CT_img.GetPixelID()))
    utils.get_2D_projections(CT_bone, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MaxIP_')
    #utils.get_2D_projections(CT_bone, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MinIP_')

    CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask,CT_img.GetPixelID()))
    utils.get_2D_projections(CT_lean, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_lean_MaxIP_')
    utils.get_2D_projections(CT_lean, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_lean_MinIP_')
    
    count += 1
    print(str(row['npr']) + '_' + str(row['scan_date']))
    print("count: ", count)
    #except: 
        #print(str(row['npr']) + '_' + str(row['scan_date']))
    #    pass

    if count > 10:
        break

                                         source_directory       patient_directory scan_date              npr
0  /media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT  lpr385705046400_SUV_CT  20140313  lpr385705046400
2  /media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT  lpr415675513429_SUV_CT  20190201  lpr415675513429
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - mean intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the SUV volume image! 
Finished generating 3 - mean intensity 2D projections from the SUV volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - min intensity 2D projections from the CT volume image! 
lpr385705046400_20140313
count:  1
Finished generating 3 - max intensity 2D projections from the CT volume im

In [18]:
resampled_directory_df.head()

,source_directory,patient_directory,scan_date,npr
0,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr865130139443_SUV_CT,20210622,npr865130139443
2,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr865130139443_SUV_CT,20211126,npr865130139443
4,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr382894159583_SUV_CT,20150127,npr382894159583
6,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr382894159583_SUV_CT,20140828,npr382894159583
8,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr717322658733_SUV_CT,20180718,npr717322658733
